In [ ]:
! pip install transformers
! pip install datasets
! pip install adapter-transformers

In [ ]:
import transformers
import torch

In [5]:
torch.cuda.is_available()

True

In [6]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving naics_desc_data_2022.xlsx to naics_desc_data_2022.xlsx


In [8]:
import io
dataset = pd.read_excel(io.BytesIO(uploaded['naics_desc_data_2022.xlsx']))

In [9]:
dataset.head()

,naics,examples
0,482112,Beltline railroads
1,482112,"Freight railways, short-line or beltline"
2,482112,Logging railroads
3,482112,"Railroad transportation, short-line or beltline"
4,482112,"Railroads, short-line or beltline"


In [10]:
sentences = dataset.examples.values
labels = dataset.naics.values

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)

LabelEncoder()

In [12]:
y_enc = le.transform(labels)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences, y_enc, test_size=0.1, random_state=42)

In [14]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [17]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df1 = pd.DataFrame({'sentences': list(X_train), 'labels': list(y_train)})
dataset = ds.dataset(pa.Table.from_pandas(df1).to_batches())

### convert to Huggingface dataset
hg_dataset1 = Dataset(pa.Table.from_pandas(df1))

In [19]:
df2 = pd.DataFrame({'sentences': list(X_test), 'labels': list(y_test)})
dataset = ds.dataset(pa.Table.from_pandas(df2).to_batches())

### convert to Huggingface dataset
hg_dataset2 = Dataset(pa.Table.from_pandas(df2))

In [20]:
import datasets
dd = datasets.DatasetDict({"train":hg_dataset1,"test":hg_dataset2})

In [22]:
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(
      batch["sentences"],
      max_length=256,
      truncation=True,
      padding="max_length"
  )

# Encode the input data
dd = dd.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
# dd = dd.rename_column("labels", "labels")
# Transform to pytorch tensors and only output the required columns
dd.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/17930 [00:00<?, ? examples/s]

Map:   0%|          | 0/1993 [00:00<?, ? examples/s]

In [24]:
from transformers import BertConfig, BertModelWithHeads

id2label = dict(list(enumerate([str(x) for x in le.classes_])))

config = BertConfig.from_pretrained(
    "bert-base-uncased",
    id2label=id2label,
)
model = BertModelWithHeads.from_pretrained(
    "bert-base-uncased",
    config=config,
)
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/bert/adapter_model.py:269: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/bert/adapter_model.py:247: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactl

BertModelWithHeads(
  (shared_parameters): ModuleDict()
  (bert): BertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (key): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (value): Linear(
                in_features=1024

In [25]:
from transformers.adapters import AdapterConfig

config = AdapterConfig(mh_adapter=True, output_adapter=True, reduction_factor=16, non_linearity="relu")
model.add_adapter("bottleneck_adapter", config=config)
model.to(device)

BertModelWithHeads(
  (shared_parameters): ModuleDict()
  (bert): BertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (key): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (value): Linear(
                in_features=1024

In [26]:
model.add_classification_head("cb", num_labels=len(id2label))
model.train_adapter("bottleneck_adapter")
model.to(device)

BertModelWithHeads(
  (shared_parameters): ModuleDict()
  (bert): BertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (key): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (value): Linear(
                in_features=1024

In [29]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    # overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dd["train"],
    eval_dataset=dd["test"],
    compute_metrics=compute_accuracy,
)

In [30]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17930
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2805
  Number of trainable parameters = 8428530


OutOfMemoryError: ignored